In [1]:
import numpy as np 
import pandas as pd 
import os
from sklearn.preprocessing import Imputer

In [2]:
data = pd.read_csv("mn_crimes_and_schools.csv")

In [3]:
data.head()

,tax_value,last_sold_price,property_size,zestimate_amount,bathrooms,zestimate_valuation_range_high,tax_year,zestimate_value_change,latitude,zestimate_percentile,...,TOTAL MISDEMEANOR OFFENSES_last_3_years,TOTAL MISDEMEANOR OFFENSES_percent_change,TOTAL NON-SEVEN MAJOR FELONY OFFENSES_last_3_years,TOTAL NON-SEVEN MAJOR FELONY OFFENSES_percent_change,TOTAL SEVEN MAJOR FELONY OFFENSES_last_3_years,TOTAL SEVEN MAJOR FELONY OFFENSES_percent_change,UNAUTHORIZED USE OF A VEHICLE_last_3_years,UNAUTHORIZED USE OF A VEHICLE_percent_change,VEHICLE AND TRAFFIC LAWS_last_3_years,VEHICLE AND TRAFFIC LAWS_percent_change
0,NaN,NaN,NaN,849427.0,1.0,1044795.0,NaN,NaN,40.724448,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881
1,1642000.0,NaN,2250.0,NaN,2.0,NaN,2015.0,NaN,40.724420,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881
2,1674000.0,NaN,2250.0,7332356.0,1.0,7992268.0,2015.0,NaN,40.724385,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881
3,NaN,NaN,NaN,1991289.0,1.0,2708153.0,NaN,NaN,40.724332,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.724845,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881


In [4]:
# get rid of rows that have missing values for our y value (zestimate_amount)
data = data[np.isfinite(data['zestimate_amount'])]

In [5]:
def clean_data (data, impute=True):
    
    # get rid of all zestimate values (besides our y), links, and Unnamed:22 (this is all NAs)
    data = data.drop(['zestimate_valuation_range_high', 'zestimate_value_change', 'zestimate_percentile', 'zestimate_last_updated',
                   'zestimate_valuationRange_low', 'graph_data_link', 'home_detail_link', 'map_this_home_link', 'Unnamed: 22',
                    'zillow_id', 'last_sold_date', 'last_sold_price'], 1)

    # convert school district and precinct to string to get dummies
    data['schooldistrict_string'] = data['schooldistrict'].astype(str)
    data['precinct_string'] = data['precinct'].astype(str)

    # replace all categorical variables with dummy variables
    dummies = pd.get_dummies(data[['home_type', 'neighborhood', 'schooldistrict_string', 'precinct_string']])
    data = pd.concat([data, dummies], axis=1)
    data = data.drop(['home_type', 'neighborhood', 'schooldistrict', 'schooldistrict_string', 'precinct', 'precinct_string'], 1)
    
    data = data.replace(np.inf, 0)
    data = data.replace(-np.inf, 0)
    
    print "Columns with Missing Values:"
    for column in data.columns:
        percent_missing = len(data[data[column] != data[column]])/float(len(data))
        if percent_missing > 0:
            print column, "\t\t\t",percent_missing

    column_names = data.columns.values
    
    if impute:
        imputer_data = Imputer(strategy = "median")
        imputer_data.fit(data)
        data = imputer_data.transform(data)
        print "Missing values imputed"
    
    data = pd.DataFrame(data, columns = column_names)
    
    return data

In [6]:
cleaned_data_imputed = clean_data (data)

Columns with Missing Values:
tax_value 			0.44906642639
property_size 			0.449817289883
bathrooms 			0.347649797267
tax_year 			0.44906642639
bedrooms 			0.33298293037
year_built 			0.326625619462
home_size 			0.22005306102
Missing values imputed


In [7]:
cleaned_data_not_imputed = clean_data (data, impute=False)

Columns with Missing Values:
tax_value 			0.44906642639
property_size 			0.449817289883
bathrooms 			0.347649797267
tax_year 			0.44906642639
bedrooms 			0.33298293037
year_built 			0.326625619462
home_size 			0.22005306102


In [8]:
test_imputed = cleaned_data_imputed.sample(n=3000)
train_imputed = cleaned_data_imputed.drop(test_imputed.index)

In [9]:
test_imputed.to_csv("test_imputed.csv", index = False)
train_imputed.to_csv("train_imputed.csv", index = False)

In [10]:
test_not_imputed = cleaned_data_not_imputed.sample(n=3000)
train_not_imputed = cleaned_data_not_imputed.drop(test_not_imputed.index)

In [11]:
test_not_imputed.to_csv("test_not_imputed.csv", index = False)
train_not_imputed.to_csv("train_not_imputed.csv", index = False)